- KBS 뉴스 url, 제목, 요약, 날짜를 출력

- AI, 헬스케어, LG, 데이터 등의 요인이 표함되어 있는 기사만 크롤링

- 추가한다면 하단의 페이지 수를 추출해서 range에 변수로 넣는것.

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By

In [30]:
# 빈 리스트 만들기
url_list = []
title_list = []
body_list = []
date_list = []

# 오늘 날짜 입력
today = datetime.today().strftime("%Y%m%d")

# 서비스 변수 설정
service = Service(executable_path=ChromeDriverManager().install())

# 1. 페이지를 돌면서 작업을 수행할 예정
for page_num in range(1,45):

    # Selenium으로 웹 드라이버를 실행
    driver = webdriver.Chrome(service = service)

    # url 전달
    url = f'https://news.kbs.co.kr/news/pc/category/category.do?ref=pSiteMap#{today}&{page_num}'
    driver.get(url)

    # 기다려달라는 값을 전달 : 최대 10초 기다리기
    wait = WebDriverWait(driver, 10)

    # 드라이버 접근 > 페이지 소스 가져오기
    html = driver.page_source

    # 드라이버 종료
    driver.quit()

    #2. BeautifulSoup 사용해서 파싱하고, 태그들을 찾아서 담아줌
    soup = BeautifulSoup(html,'html.parser')
    box_contents = soup.find_all('a', class_ = 'box-content flex-style')

    keywords = ['GPT', 'AI', '데이터', 'LG', '헬스케어', '인테리어']

    #3. 박스 데이터들을 하나씩 돌면서 리스트에 데이터를 담음
    for box_content in box_contents:
        url = "http://news.kbs.co.kr" + box_content['href']
        title = box_content.find('p',class_='title').text
        body = box_content.find('p',class_='news-text').text
        date = box_content.find('span', class_='date').text

        # keyword가 있는 경우만 저장
        for keyword in keywords:
            if title.find(keyword) == -1 and body.find(keyword) == -1:
                continue
            else:
                url_list.append(url)
                title_list.append(title)
                body_list.append(body)
                date_list.append(date)
    
# 데이터 저장
data = {'뉴스url':url_list, '제목':title_list, '내용':body_list, '날짜':date_list}
df = pd.DataFrame(data)
df

df.to_csv('news_kbs_pagenation.csv', index = False)

In [32]:
df

,뉴스url,제목,내용,날짜
0,http://news.kbs.co.kr/news/view.do?ncd=7987875,"미중, 2차 1.5트랙 대화…“충돌 피하고 무역·AI협력 심화 공감”",전략 경쟁 격화 속에서도 부문별 소통 채널을 속속 활성화 중인 미국과 중국이 1.5...,2024.06.14 (11:00)
